<a href="https://colab.research.google.com/github/AmitNaikRepository/my-projects/blob/master/Text_generationWith_GPT_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import json
import os
import numpy as np
import tensorflow as tf

In [2]:
!git clone https://github.com/openai/gpt-2.git

Cloning into 'gpt-2'...
remote: Enumerating objects: 221, done.
remote: Total 221 (delta 0), reused 0 (delta 0), pack-reused 221
Receiving objects: 100% (221/221), 4.37 MiB | 6.21 MiB/s, done.
Resolving deltas: 100% (117/117), done.


In [0]:

import os
os.chdir("gpt-2")

In [4]:
!pip3 install tensorflow-gpu==1.12.0


     |████████████████████████████████| 281.7MB 27kB/s 
     |████████████████████████████████| 3.1MB 43.0MB/s 
ERROR: tensorflow 1.15.0 has requirement tensorboard<1.16.0,>=1.15.0, but you'll have tensorboard 1.12.2 which is incompatible.
  Found existing installation: tensorboard 1.15.0
    Uninstalling tensorboard-1.15.0:
      Successfully uninstalled tensorboard-1.15.0


In [5]:

!pip3 install -r requirements.txt

     |████████████████████████████████| 81kB 5.6MB/s 
     |████████████████████████████████| 604kB 27.9MB/s 
     |████████████████████████████████| 51kB 6.6MB/s 
  Created wheel for fire: filename=fire-0.2.1-py2.py3-none-any.whl size=103527 sha256=56940c8dfd1f7c0109c193ac910d5ab4efb0f9d178398c551cbc1793f8d76d82
  Stored in directory: /root/.cache/pip/wheels/31/9c/c0/07b6dc7faf1844bb4688f46b569efe6cafaa2179c95db821da
  Created wheel for regex: filename=regex-2017.4.5-cp36-cp36m-linux_x86_64.whl size=533164 sha256=ce6ca5030b08cd22a5f4340fb1f16cf77341591629a921bd7ad1923667e2f42d
  Stored in directory: /root/.cache/pip/wheels/75/07/38/3c16b529d50cb4e0cd3dbc7b75cece8a09c132692c74450b01
Successfully built fire regex
  Found existing installation: tqdm 4.28.1
    Uninstalling tqdm-4.28.1:
      Successfully uninstalled tqdm-4.28.1


In [6]:

!python3 download_model.py 345M

Fetching checkpoint: 1.00kit [00:00, 705kit/s]                                                      
Fetching encoder.json: 1.04Mit [00:00, 53.9Mit/s]                                                   
Fetching hparams.json: 1.00kit [00:00, 695kit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 1.42Git [00:27, 52.2Mit/s]                                 
Fetching model.ckpt.index: 11.0kit [00:00, 6.84Mit/s]                                               
Fetching model.ckpt.meta: 927kit [00:00, 57.6Mit/s]                                                 
Fetching vocab.bpe: 457kit [00:00, 48.2Mit/s]                                                       


In [0]:
!export PYTHONIOENCODING=UTF-8

In [0]:
os.chdir('src')

In [0]:
import model,sample,encoder

In [0]:
def interact_model(
    model_name,
    seed,
    nsamples,
    batch_size,
    length,
    temperature,
    top_k,
    models_dir
):
    models_dir = os.path.expanduser(os.path.expandvars(models_dir))
    if batch_size is None:
        batch_size = 1
    assert nsamples % batch_size == 0

    enc = encoder.get_encoder(model_name, models_dir)
    hparams = model.default_hparams()
    with open(os.path.join(models_dir, model_name, 'hparams.json')) as f:
        hparams.override_from_dict(json.load(f))

    if length is None:
        length = hparams.n_ctx // 2
    elif length > hparams.n_ctx:
        raise ValueError("Can't get samples longer than window size: %s" % hparams.n_ctx)

    with tf.Session(graph=tf.Graph()) as sess:
        context = tf.placeholder(tf.int32, [batch_size, None])
        np.random.seed(seed)
        tf.set_random_seed(seed)
        output = sample.sample_sequence(
            hparams=hparams, length=length,
            context=context,
            batch_size=batch_size,
            temperature=temperature, top_k=top_k
        )

        saver = tf.train.Saver()
        ckpt = tf.train.latest_checkpoint(os.path.join(models_dir, model_name))
        saver.restore(sess, ckpt)

        while True:
            raw_text = input("Model prompt >>> ")
            while not raw_text:
                print('Prompt should not be empty!')
                raw_text = input("Model prompt >>> ")
            context_tokens = enc.encode(raw_text)
            generated = 0
            for _ in range(nsamples // batch_size):
                out = sess.run(output, feed_dict={
                    context: [context_tokens for _ in range(batch_size)]
                })[:, len(context_tokens):]
                for i in range(batch_size):
                    generated += 1
                    text = enc.decode(out[i])
                    print("=" * 40 + " SAMPLE " + str(generated) + " " + "=" * 40)
                    print(text)
            print("=" * 80)


In [12]:
interact_model(
    '345M',
    None,
    1,
    1,
    300,
    1,
    0,
    '/content/gpt-2/models'
)





Instructions for updating:
Use `tf.cast` instead.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Use `tf.random.categorical` instead.
INFO:tensorflow:Restoring parameters from /content/gpt-2/models/345M/model.ckpt
Model prompt >>> i have meet akshay in 91springboard he is web developer i really like his work  
======================================== SAMPLE 1 ========================================
  22 jil jaffrey ander249 loves coconut shell componenosts . |ttenz how ahn i eaiik mein magaam little bruned gnome alios for slender indicates euiam haaz


127 overland ski mountains cody keeps me in us clouns Now we are aiming at 30 topsolutions :


128 Noota nakers nops Posts: 61 Joined: 08 Feb 2011 Home Dinger posts:


37= byhp lame outlook caz properly run australia A picture some one took at Adler miles is real:Sep 19th 1988<|endoftext|>Description

**Editor's Note** Round 1! Havocettes is streaming a second

KeyboardInterrupt: ignored